# OpenFermion Gran Canonical

A diferencia de FermionicML, donde se trabaja en el canónico, la idea es implementar las matrices densidad de 1 y 2 cuerpos en el ensamble canónico, utilizando para ello operadores dados por OpenFermion, por la facilidad de generar los operadores (rhoKarrays) tomando la get_sparse_operator sobre la base. 

In [56]:
# CONFIG
d = 4 # número de niveles

In [96]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated

# Generación de base

class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)
    
    def idx_to_repr(self, idx):
        return self.canonicals[idx]
    
    def opr_to_idx(self, opr):
        return self.base.index(opr)

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None):
        basis = []
        num_ele = []
        for k in reversed(range(0,2**d)):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    oper = self.bin_to_op(b)
                    basis.append(oper)
                    num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def __init__(self, d, num = None):
        self.d = d
        self.num = num
        self.base, self.num_ele = self.create_basis(d, num)
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        
    @staticmethod
    def bdb(i, j):
        return of.FermionOperator(((i,1),(j,0)))

def rho_1_arrays_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d

    for i in tqdm(range(0, d)):
        for j in range(0, d):
            op = basis.bdb(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat



In [101]:
basis = fixed_basis(d, num = 2)
print(basis.base)
print(basis.num_ele)
rho_1_arrays_gen(basis)[1,0]

[1.0 [0^ 1^], 1.0 [0^ 2^], 1.0 [0^ 3^], 1.0 [1^ 2^], 1.0 [1^ 3^], 1.0 [2^ 3^]]
[12, 10, 9, 6, 5, 3]


100%|██████████| 4/4 [00:00<00:00, 32.48it/s]


array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [ ]:

#bbb = fixed_basis(d).rho_1_arrays
i,j = 1, 1
base = fixed_basis(d).base
cano = fixed_basis(d).canonicals
OPNUM = 5
wfc = base[OPNUM]
idx = cano[OPNUM]
print(idx, wfc)
OPME = fixed_basis.bdb(j, i)
print(of.get_sparse_operator(OPME, n_qubits=d))

#print(wfc)
mean_val = lambda x: np.real(np.transpose(idx) @ of.get_sparse_operator(x, n_qubits=d) @ idx)

print(mean_val(fixed_basis.bdb(j, i)))
#print(fixed_basis.bdb(j, i, d))
#print(op.get_sparse_operator(fixed_basis.bdb(j, i, d), n_qubits=d))
#op.linalg.expectation(fixed_basis.bdb(j, i, d), state=op.get_sparse_operator(base[1], n_qubits=d))
print(base[6], base[5])
print(fixed_basis(d).mean_val(wfc, OPME))